In [1]:
from bs4 import BeautifulSoup as bs
import os
import re
import json

In [2]:
# get working directory, move up one layer
path = os.getcwd()
parDir = path.replace('Scripts','')

In [3]:
# list all root, directories, and files
rootCollector=[]
dirsCollector=[]
filesCollector=[]
for (root, dirs, files) in  os.walk(parDir):
    rootCollector.append(root)
    dirsCollector.append(dirs)
    filesCollector.append(files)

In [4]:
rootCollector[76]

'd:\\Work\\Repo\\R Graph\\BB Request'

In [5]:
filesCollector[76]

['BB Dataset Maker.ipynb', 'network graph BB.R']

In [6]:
# identify html files
idxCollector = []
filepathCollector = []
for i in range(len(rootCollector)):
    root = rootCollector[i]
    files = filesCollector[i]
    for file in files:
        if file.endswith(".html"):
            filepath = root+'\\'+file
            filepathCollector.append(filepath)
            idxCollector.append(i)

In [7]:
idxCollector

[77, 77, 77, 108, 108, 122, 122, 122, 122, 129, 129, 130, 130, 139, 139]

In [8]:
filepathCollector

['d:\\Work\\Repo\\R Graph\\BB Request\\Graph 2208\\result_BB_Fundingv1_External.html',
 'd:\\Work\\Repo\\R Graph\\BB Request\\Graph 2208\\result_BB_Lendingv1_External.html',
 'd:\\Work\\Repo\\R Graph\\BB Request\\Graph 2208\\result_Graph 2208v1.2_External.html',
 'd:\\Work\\Repo\\R Graph\\BB Request\\Graph 2212\\result_Graph 2212v1_BB Funding 50.html',
 'd:\\Work\\Repo\\R Graph\\BB Request\\Graph 2212\\result_Graph 2212v1_BB Lending 50.html',
 'd:\\Work\\Repo\\R Graph\\GB Request\\Graph 2208\\result_Graph 2208v1_GB Funding 100.html',
 'd:\\Work\\Repo\\R Graph\\GB Request\\Graph 2208\\result_Graph 2208v1_GB Lending All.html',
 'd:\\Work\\Repo\\R Graph\\GB Request\\Graph 2208\\result_Graph 2208v1_GB_Top 50 Funding.html',
 'd:\\Work\\Repo\\R Graph\\GB Request\\Graph 2208\\result_Graph 2208v1_GB_Top 50 Lending.html',
 'd:\\Work\\Repo\\R Graph\\RSME Request\\result_Graph 2208v1_RSME_Funding.html',
 'd:\\Work\\Repo\\R Graph\\RSME Request\\result_Graph 2208v1_RSME_Lending.html',
 'd:\\Work\\R

In [9]:
def masker(name):
    words = name.split()
    masked_name = []
    
    for word in words:
        if len(word) > 2:  # Words with more than 2 characters
            masked_word = word[0] + '*' * (len(word) - 2) + word[-1]
        else:
            masked_word = word  # Keep words with 1 or 2 characters unchanged
        
        masked_name.append(masked_word)
    
    return ' '.join(masked_name)

In [12]:
for html in filepathCollector:
    # Step 1: Load the HTML file
    with open(html, 'r') as file:
        html_content = file.read()

    # Step 2: Parse the HTML using BeautifulSoup
    soup = bs(html_content, 'html.parser')

    # Step 3: Find the script tag containing JSON data
    script_tag = soup.find('script', {'type': 'application/json'})

    if script_tag:
        # Step 4: Extract the JSON string
        json_string = script_tag.string

        # Step 5: Parse the JSON string
        data = json.loads(json_string)
    else:
        print("JSON data not found in the HTML file.")

    # create new label
    try:
        # creating new label
        labels = data['x']['nodes']['label']
        newLabels = [masker(i) for i in labels] #new labels done

        # replace old labels with new
        data['x']['nodes']['label'] = newLabels

        # create name dict
        nameDict = dict(zip(labels, newLabels))
    except:
        pass

    # create new title
    try:
        titles = data['x']['nodes']['title']
        newTitles = []
        for title in titles:
            titleSplitted = title.split('|')
            titleSplitted[0] = masker(titleSplitted[0])+' '
            newTitles.append('|'.join(titleSplitted))
        data['x']['nodes']['title']=newTitles
    except:
        pass

    # create new entity
    try:
        entities = data['x']['nodes']['entity']
        newEntities = []
        for entity in entities:
            newEntities.append(masker(entity))
        data['x']['nodes']['entity']=newEntities
    except:
        pass

    # create new values
    try:
        values = data['x']['byselection']['values']
        newValues = []
        for value in values:
            newValues.append(masker(value))
        data['x']['byselection']['values']=newValues
    except:
        pass

    # rvert data back to json
    newData = json.dumps(data)

    # write back the json
    soup.find('script', {'type': 'application/json'}).string=newData

    with open(html, 'w') as file:
        file.write(str(soup))